In [7]:
%load_ext autoreload
%autoreload 2
import time
from quantum_spins import QuantumSpinSystem
import sciexplorer.physics_run_quantumspins as prq
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
sim_q = QuantumSpinSystem(**prq.models['DYNAMICS_ARBITRARY_THREE_SPINS_TWO_SPINS_ACCESS'])

In [9]:
Htest = '''
A = 1.
hx = 0.6
Jz = 0.6
N = len(Sx)
H = 0*Sx[0]
# Nearest-neighbor Ising coupling in z
for i in range(N-1):
    H = H - Jz * (Sz[i] @ Sz[i+1])
# Three-body cluster term with tunable coefficient A
for i in range(1, N-1):
    H = H + A * (Sz[i-1] @ Sx[i] @ Sz[i+1])
# Uniform transverse field along x
for i in range(N):
    H = H - hx * Sx[i]'''
sim_q.solver.set_Hamiltonian("Htest",Htest)

{'result_str': 'Success: Set Hamiltonian successfully and saved it under the label: Htest.'}

In [10]:
sim_q.solver.Hamiltonian["Htest"]

Array([[-1.2       , -0.6       ,  0.39999998,  0.        , -0.6       ,
         0.        ,  0.        ,  0.        ],
       [-0.6       ,  0.        ,  0.        , -1.6       ,  0.        ,
        -0.6       ,  0.        ,  0.        ],
       [ 0.39999998,  0.        ,  1.2       , -0.6       ,  0.        ,
         0.        , -0.6       ,  0.        ],
       [ 0.        , -1.6       , -0.6       ,  0.        ,  0.        ,
         0.        ,  0.        , -0.6       ],
       [-0.6       ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.6       , -1.6       ,  0.        ],
       [ 0.        , -0.6       ,  0.        ,  0.        , -0.6       ,
         1.2       ,  0.        ,  0.39999998],
       [ 0.        ,  0.        , -0.6       ,  0.        , -1.6       ,
         0.        ,  0.        , -0.6       ],
       [ 0.        ,  0.        ,  0.        , -0.6       ,  0.        ,
         0.39999998, -0.6       , -1.2       ]], dtype=float32)

In [11]:
N=5
H1 = f"""
A=0.
N={N}
H=0*Sz[0]
# for j in range(N-1):
#     H+=A*Sz[j] @ Sz[j+1]
for j in range(N):
    H+=Sz[j]"""

H2 = f"""
A= 0.4
#B=0.9
N={N}
H=0*Sz[0]
for j in range(N):
    H+=Sz[j] 
for j in range(N-1):
    H+=A*Sx[j] @ Sx[j+1]
H+=A*Sx[0] @ Sx[N-1]
H *= 2
"""


sim_q.set_N(N)

r1 = sim_q.solver.set_Hamiltonian("H1",H1)
r2 = sim_q.solver.set_Hamiltonian("H2",H2)
print(r1, r2)

H1_c = sim_q.solver.Hamiltonian["H1"]
H2_c = sim_q.solver.Hamiltonian["H2"]

fid = sim_q.fidelity_for_Hamiltonians(H1_c, H2_c)
#fid = sim_q.fidelity_for_ground_states(H1_c, H2_c)

print(fid, fid**(1/N))

{'result_str': 'Success: Set Hamiltonian successfully and saved it under the label: H1.'} {'result_str': 'Success: Set Hamiltonian successfully and saved it under the label: H2.'}
0.43103433 0.84508884


In [12]:
# A = 0.4
# H1 = f"""M=3
# N=M**2

# def idx(jx,jy,M):
#     return (jx+M)%M + ((jy+M)%M)*M

# H=0*Sz[0]
# for jx in range(M):
#     for jy in range(M):
#         for dx,dy in [(-1,0),(+1,0),(0,+1),(0,-1)]:
#             H+={A}*(Sx[idx(jx,jy,M)] @ Sx[idx(jx+dx,jy+dy,M)] + Sy[idx(jx,jy,M)] @ Sy[idx(jx+dx,jy+dy,M)] + Sz[idx(jx,jy,M)] @ Sz[idx(jx+dx,jy+dy,M)])
# for j in range(N):
#     H-=2*Sx[j]"""


# H2 = f"""M=3
# N=M**2

# def idx(jx,jy,M):
#     return (jx+M)%M + ((jy+M)%M)*M

# H=0*Sz[0]
# for jx in range(M):
#     for jy in range(M):
#         for dx,dy in [(-1,0),(+1,0),(0,+1),(0,-1)]:
#             H+={A}*(Sx[idx(jx,jy,M)] @ Sx[idx(jx+dx,jy+dy,M)] + Sy[idx(jx,jy,M)] @ Sy[idx(jx+dx,jy+dy,M)] + Sz[idx(jx,jy,M)] @ Sz[idx(jx+dx,jy+dy,M)])
# for j in range(N):
#     H-=Sx[j]"""

In [14]:
H2 = f"""M=3
N=M**2

def idx(jx,jy,M):
    return (jx+M)%M + ((jy+M)%M)*M

H=0*Sz[0]
for jx in range(M):
    for jy in range(M):
        for dx,dy in [(-1,0),(+1,0),(0,+1),(0,-1)]:
            H+=A*(Sx[idx(jx,jy,M)] @ Sx[idx(jx+dx,jy+dy,M)] + Sy[idx(jx,jy,M)] @ Sy[idx(jx+dx,jy+dy,M)] + Sz[idx(jx,jy,M)] @ Sz[idx(jx+dx,jy+dy,M)])
for j in range(N):
    H-=2*Sx[j]"""

sim_q.announce_Hamiltonian(H2)
sim_q.last_result

setHamiltonian: Error in executing provided code: name 'A' is not defined
Error: 'agentHamiltonian'


{'fidelity': nan,
 'N': 5,
 'error': "'agentHamiltonian'",
 'evals': '100',
 'Hamiltonian': 'M=3\nN=M**2\n\ndef idx(jx,jy,M):\n    return (jx+M)%M + ((jy+M)%M)*M\n\nH=0*Sz[0]\nfor jx in range(M):\n    for jy in range(M):\n        for dx,dy in [(-1,0),(+1,0),(0,+1),(0,-1)]:\n            H+=A*(Sx[idx(jx,jy,M)] @ Sx[idx(jx+dx,jy+dy,M)] + Sy[idx(jx,jy,M)] @ Sy[idx(jx+dx,jy+dy,M)] + Sz[idx(jx,jy,M)] @ Sz[idx(jx+dx,jy+dy,M)])\nfor j in range(N):\n    H-=2*Sx[j]'}

In [15]:
Ha = """
H=Sz[0] * 0
for j in range(N-1):
    H+=A*Sz[j] @ Sz[j+1]
for j in range(N):
    H-=Sx[j]*0.1
"""
sim_q.announce_Hamiltonian(Ha)
sim_q.last_result

setHamiltonian: Error in executing provided code: name 'N' is not defined
Error: 'agentHamiltonian'


{'fidelity': nan,
 'N': 5,
 'error': "'agentHamiltonian'",
 'evals': '100',
 'Hamiltonian': '\nH=Sz[0] * 0\nfor j in range(N-1):\n    H+=A*Sz[j] @ Sz[j+1]\nfor j in range(N):\n    H-=Sx[j]*0.1\n'}